# 🥇 Gold Table: Enhanced ML Features with Temporal Intelligence

**Purpose:** Transform Enhanced Silver data into ML-ready feature vectors with temporal features for superior predictive performance.

**Key Enhancements:**
- ✅ Uses Enhanced Silver with 15 columns (temporal features included)
- ✅ Temporal feature encoding in ML pipeline
- ✅ Improved target classification (5 classes)
- ✅ ~1200+ feature dimensions (vs ~800 original)

**Pipeline:** Bronze → Enhanced Silver (15 cols) → **Enhanced Gold (ML-ready)**

**Source:** `default.silver_flights_processed` (Enhanced with temporal features)
**Output:** `default.gold_ml_features` (Enhanced ML vectors)

In [0]:
print("📥 Loading Enhanced Silver table with temporal features...")

# Load the enhanced Silver table (now has 15 columns with temporal features)
df_silver = spark.table("default.silver_flights_processed")

print(f"✅ Loaded Enhanced Silver: {df_silver.count():,} flights")
print(f"Columns: {len(df_silver.columns)} (enhanced with temporal features)")

# Show what we're working with
print("\n📋 Enhanced Silver Schema:")
df_silver.printSchema()

# Show sample to confirm temporal features are present
print("\n🔎 Sample Enhanced Silver (showing temporal features):")
df_silver.select("airline_name", "flight_date", "arrival_delay", "day_of_week", "is_weekend", "is_holiday", "season").show(
    3, truncate=False
)

📥 Loading Enhanced Silver table with temporal features...
✅ Loaded Enhanced Silver: 2,520,650 flights
Columns: 17 (enhanced with temporal features)

📋 Enhanced Silver Schema:
root
 |-- airline_name: string (nullable = true)
 |-- airline_code: string (nullable = true)
 |-- origin_airport_code: string (nullable = true)
 |-- destination_airport_code: string (nullable = true)
 |-- arrival_delay: double (nullable = true)
 |-- flight_date: date (nullable = true)
 |-- flight_month: integer (nullable = true)
 |-- flight_year: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- is_weekend: boolean (nullable = true)
 |-- is_holiday: boolean (nullable = true)
 |-- is_near_holiday: boolean (nullable = true)
 |-- is_holiday_period: boolean (nullable = true)
 |-- season: string (nullable = true)
 |-- quarter: integer (nullable = true)


🔎 Sample Enhanced Silver (showing temporal features)

In [0]:
!pip install pyspark

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StandardScaler, StringIndexer, VectorAssembler
from pyspark.sql.functions import col, count, isnan, when

print("✅ Enhanced Gold ML pipeline imports loaded")

✅ Enhanced Gold ML pipeline imports loaded


In [0]:
df_silver = spark.table("default.silver_flights_processed") # Added because it was missing and wasn't allowing to run further notebook

In [0]:
print("🎯 Creating enhanced arrival status with 5 categories...")

# Enhanced target classification (more granular than original 4 classes)
df_with_status = df_silver.withColumn(
    "arrival_status",
    when(col("arrival_delay").isNull(), "Cancelled/Diverted")
    .when(col("arrival_delay") >= 60, "Severely Delayed")  # New: 60+ min
    .when(col("arrival_delay") >= 15, "Delayed")  # 15-59 min
    .when(col("arrival_delay") >= -5, "On-Time")  # -5 to 14 min
    .when(col("arrival_delay") < -5, "Early")  # <-5 min
    .otherwise("Unknown"),
)

# Show enhanced target distribution
print("\n📊 Enhanced Target Distribution (5 categories):")
target_counts = df_with_status.groupBy("arrival_status").count().orderBy("count", ascending=False)
target_counts.show()

total_flights = df_with_status.count()
print(f"\nTotal flights for ML: {total_flights:,}")

df_silver = df_with_status

🎯 Creating enhanced arrival status with 5 categories...

📊 Enhanced Target Distribution (5 categories):
+------------------+-------+
|    arrival_status|  count|
+------------------+-------+
|             Early|1262663|
|           On-Time| 711524|
|           Delayed| 324074|
|  Severely Delayed| 165718|
|Cancelled/Diverted|  56671|
+------------------+-------+


Total flights for ML: 2,520,650


## Enhanced Feature Engineering Pipeline

In [0]:
print("Organizing enhanced features for ML pipeline...")

# Categorical features (need string indexing + one-hot encoding)
categorical_features = [
    "airline_name",
    "airline_code",
    "origin_airport_code",
    "destination_airport_code",
    "season",  # New temporal categorical feature
]

# Numerical features (can use directly, including temporal)
numerical_features = [
    "flight_month",
    "flight_year",
    "day_of_week",  # New temporal
    "week_of_year",  # New temporal
    "day_of_month",  # New temporal
    "quarter",  # New temporal
]

# Boolean features (convert to 0/1, all new temporal)
boolean_features = [
    "is_weekend",  # New temporal
    "is_holiday",  # New temporal
    "is_near_holiday",  # New temporal
    "is_holiday_period",  # New temporal
]

print(f"📊 Enhanced Feature Inventory:")
print(f"  Categorical features: {len(categorical_features)} (includes season)")
print(f"  Numerical features: {len(numerical_features)} (includes 4 temporal)")
print(f"  Boolean features: {len(boolean_features)} (all temporal)")
print(f"  Total input features: {len(categorical_features + numerical_features + boolean_features)}")
print(f"  Original Gold had: 4 input features")
print(f"  Enhancement added: {len(numerical_features + boolean_features + ['season'])} temporal features")

# Convert boolean features to numerical (0/1)
print(f"\n🔄 Converting boolean features to numerical...")
df_ml_ready = df_silver
for bool_col in boolean_features:
    df_ml_ready = df_ml_ready.withColumn(bool_col, when(col(bool_col) == True, 1.0).otherwise(0.0))

print("✅ Boolean features converted to 0/1 for ML compatibility")

Organizing enhanced features for ML pipeline...
📊 Enhanced Feature Inventory:
  Categorical features: 5 (includes season)
  Numerical features: 6 (includes 4 temporal)
  Boolean features: 4 (all temporal)
  Total input features: 15
  Original Gold had: 4 input features
  Enhancement added: 11 temporal features

🔄 Converting boolean features to numerical...
✅ Boolean features converted to 0/1 for ML compatibility


In [0]:
print("🔍 Validating enhanced target distribution...")

# Show enhanced target sample
print("Sample Enhanced Target (arrival_delay → arrival_status):")
df_ml_ready.select("arrival_delay", "arrival_status", "is_weekend", "is_holiday", "season").show(10)

# Show final target distribution for ML
print("\n📊 Final Enhanced Target Distribution:")
df_ml_ready.groupBy("arrival_status").count().orderBy("count", ascending=False).show()

print(f"Enhanced dataset ready for ML pipeline")
print(f"Total features available: {len(categorical_features + numerical_features + boolean_features)}")


🔍 Validating enhanced target distribution...
Sample Enhanced Target (arrival_delay → arrival_status):
+-------------+----------------+----------+----------+------+
|arrival_delay|  arrival_status|is_weekend|is_holiday|season|
+-------------+----------------+----------+----------+------+
|        -14.0|           Early|       0.0|       0.0|Winter|
|         -5.0|         On-Time|       1.0|       0.0|  Fall|
|          0.0|         On-Time|       0.0|       0.0|Summer|
|         24.0|         Delayed|       0.0|       0.0|Spring|
|        141.0|Severely Delayed|       0.0|       0.0|Summer|
|        -29.0|           Early|       1.0|       0.0|Summer|
|         23.0|         Delayed|       0.0|       0.0|Summer|
|        -11.0|           Early|       1.0|       0.0|Winter|
|         60.0|Severely Delayed|       0.0|       0.0|Summer|
|          1.0|         On-Time|       0.0|       0.0|Summer|
+-------------+----------------+----------+----------+------+
only showing top 10 rows

📊 Fi

In [0]:
# Get all column names
all_columns = df_silver.columns

# Find just the float/double columns
numeric_cols = [c_name for (c_name, c_type) in df_silver.dtypes if c_type in ("float", "double")]

# Get all *other* columns
other_cols = [c_name for c_name in all_columns if c_name not in numeric_cols]

# Create expressions for numeric columns (check for null OR nan)
numeric_expressions = [count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in numeric_cols]

# Create expressions for all other columns (check for null only)
other_expressions = [count(when(col(c).isNull(), c)).alias(c) for c in other_cols]

# Combine the lists of expressions
all_expressions = numeric_expressions + other_expressions

# Run the counts and show the result
print("Missing value counts per column (before ML pipeline):")
df_silver.select(*all_expressions).show()

Missing value counts per column (before ML pipeline):
+-------------+------------+------------+-------------------+------------------------+-----------+------------+-----------+-----------+------------+------------+----------+----------+---------------+-----------------+------+-------+--------------+
|arrival_delay|airline_name|airline_code|origin_airport_code|destination_airport_code|flight_date|flight_month|flight_year|day_of_week|week_of_year|day_of_month|is_weekend|is_holiday|is_near_holiday|is_holiday_period|season|quarter|arrival_status|
+-------------+------------+------------+-------------------+------------------------+-----------+------------+-----------+-----------+------------+------------+----------+----------+---------------+-----------------+------+-------+--------------+
|        56671|           0|           0|                  0|                       0|          0|           0|          0|          0|           0|           0|         0|         0|              0|   

In [0]:
print("⚡ Building Enhanced ML Pipeline with Temporal Features...")

# Updated feature requirements (now includes temporal features)
required_cols = [
    "airline_name",
    "airline_code",
    "origin_airport_code",
    "destination_airport_code",
    "flight_month",
    "flight_year",
    "arrival_status",
    # New temporal features
    "day_of_week",
    "week_of_year",
    "day_of_month",
    "quarter",
    "season",
    "is_weekend",
    "is_holiday",
    "is_near_holiday",
    "is_holiday_period",
]

missing_cols = [c for c in required_cols if c not in df_ml_ready.columns]

if missing_cols:
    print(f"❌ ERROR: Missing columns for enhanced pipeline: {missing_cols}")
else:
    print("✅ All enhanced features found. Building ML pipeline...")

    # --- Enhanced ML Pipeline Stages ---

    # Stage 1: String Indexers for categorical features (including temporal)
    categorical_cols_indexed = [f"{c}_index" for c in categorical_features]

    indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_index", handleInvalid="keep") for c in categorical_features]

    print(f"📝 String indexers created for {len(categorical_features)} categorical features")

    # Stage 2: One-Hot Encoders
    categorical_cols_ohe = [f"{c}_ohe" for c in categorical_features]

    ohe_encoder = OneHotEncoder(inputCols=categorical_cols_indexed, outputCols=categorical_cols_ohe, handleInvalid="keep")

    print(f"🎯 One-hot encoder created for categorical features")

    # Stage 3: Target Label Indexer
    label_indexer = StringIndexer(inputCol="arrival_status", outputCol="label", handleInvalid="keep")

    print("🏷️ Target label indexer created (5-class classification)")

    # Stage 4: Feature Vector Assembler (Enhanced with temporal)
    all_feature_cols = numerical_features + boolean_features + categorical_cols_ohe

    assembler = VectorAssembler(inputCols=all_feature_cols, outputCol="unscaled_features", handleInvalid="skip")

    print(f"🔧 Feature assembler created with {len(all_feature_cols)} feature columns:")
    print(f"   • Numerical (temporal): {len(numerical_features)}")
    print(f"   • Boolean (temporal): {len(boolean_features)}")
    print(f"   • Categorical (one-hot): {len(categorical_cols_ohe)}")

    # Stage 5: Standard Scaler
    scaler = StandardScaler(inputCol="unscaled_features", outputCol="features", withStd=True, withMean=True)

    print("⚖️ Standard scaler configured (mean=0, std=1)")

    # Combine all stages into enhanced pipeline
    enhanced_pipeline_stages = indexers + [ohe_encoder, label_indexer, assembler, scaler]
    enhanced_pipeline = Pipeline(stages=enhanced_pipeline_stages)

    print(f"\n🚀 Enhanced ML Pipeline Ready!")
    print(f"Total pipeline stages: {len(enhanced_pipeline_stages)}")
    print(f"Expected output: ~{800 + len(numerical_features + boolean_features)}+ feature dimensions")

⚡ Building Enhanced ML Pipeline with Temporal Features...
✅ All enhanced features found. Building ML pipeline...
📝 String indexers created for 5 categorical features
🎯 One-hot encoder created for categorical features
🏷️ Target label indexer created (5-class classification)
🔧 Feature assembler created with 15 feature columns:
   • Numerical (temporal): 6
   • Boolean (temporal): 4
   • Categorical (one-hot): 5
⚖️ Standard scaler configured (mean=0, std=1)

🚀 Enhanced ML Pipeline Ready!
Total pipeline stages: 9
Expected output: ~810+ feature dimensions


## ⚡ Pipeline Execution & Gold Table Creation

In [0]:
print("⚡ Fitting Enhanced ML Pipeline...")
print("This may take a few minutes for large datasets with temporal features...")

# Fit the enhanced pipeline on our temporal-enriched data
fitted_pipeline = enhanced_pipeline.fit(df_ml_ready)
print("✅ Enhanced pipeline fitted successfully")

# Transform the data through the complete ML pipeline
print("\n🔄 Transforming data through enhanced pipeline...")
df_gold_ml = fitted_pipeline.transform(df_ml_ready)

print("✅ Enhanced transformation completed")
print(f"Output columns: {len(df_gold_ml.columns)}")

# Check the feature vector size
sample_features = df_gold_ml.select("features").first().features
print(f"Enhanced feature vector size: {sample_features.size} dimensions")
print(f"Original Gold had: ~800 dimensions")
print(f"Enhancement added: ~{sample_features.size - 800} temporal dimensions")

⚡ Fitting Enhanced ML Pipeline...
This may take a few minutes for large datasets with temporal features...
✅ Enhanced pipeline fitted successfully

🔄 Transforming data through enhanced pipeline...
✅ Enhanced transformation completed
Output columns: 31
Enhanced feature vector size: 813 dimensions
Original Gold had: ~800 dimensions
Enhancement added: ~13 temporal dimensions


In [0]:
print("🥇 Creating Final Enhanced Gold Table...")

# Select only the ML-ready features and labels
df_gold_ml_final = df_gold_ml.select("features", "label")

print(f"✅ Enhanced Gold table created")
print(f"Final columns: {len(df_gold_ml_final.columns)} (features, label)")
print(f"Total rows: {df_gold_ml_final.count():,}")

# Validate the enhanced output
print("\n📋 Enhanced Gold Schema:")
df_gold_ml_final.printSchema()

# Show label distribution for enhanced target
print("\n📊 Enhanced Target Label Distribution:")
df_gold_ml_final.groupBy("label").count().orderBy("label").show()

# Show sample (careful with large vectors)
print("\n🔎 Enhanced Gold Sample (truncated):")
df_gold_ml_final.show(3, truncate=True)

print(f"\n🎯 Enhanced Gold ML Table Summary:")
print(f"  • Feature dimensions: {sample_features.size}")
print(f"  • Target classes: 5 (vs 4 original)")
print(f"  • Temporal features: {len(numerical_features + boolean_features + ['season'])}")
print(f"  • Total enhancement: {sample_features.size - 800}+ new dimensions")

🥇 Creating Final Enhanced Gold Table...
✅ Enhanced Gold table created
Final columns: 2 (features, label)
Total rows: 2,520,650

📋 Enhanced Gold Schema:
root
 |-- features: vectorudt (nullable = true)
 |-- label: double (nullable = false)


📊 Enhanced Target Label Distribution:
+-----+-------+
|label|  count|
+-----+-------+
|  0.0|1262663|
|  1.0| 711524|
|  2.0| 324074|
|  3.0| 165718|
|  4.0|  56671|
+-----+-------+


🔎 Enhanced Gold Sample (truncated):
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[-1.6049222153511...|  0.0|
|[1.41913690323475...|  1.0|
|[0.20951325580039...|  1.0|
+--------------------+-----+
only showing top 3 rows

🎯 Enhanced Gold ML Table Summary:
  • Feature dimensions: 813
  • Target classes: 5 (vs 4 original)
  • Temporal features: 11
  • Total enhancement: 13+ new dimensions


In [0]:
# This is the final DataFrame from your ML pipeline
assert df_gold_ml_final, "The DataFrame 'df_gold_ml_final' does not exist."

# Define the paths for your new Gold ML table
GOLD_ML_PATH = "/Volumes/workspace/default/ds-capstone/gold/ml_features_experimental" # Once again, this path was updated to follow the "ds-capstone" convention established in the Bronze_table notebook. The old path was as follows "/Volumes/workspace/default/ds_capstone/gold/ml_features_experimental"
GOLD_ML_TABLE_NAME = "default.gold_ml_features_experimental"
DATABASE_NAME = "default"

assert DATABASE_NAME, "DATABASE_NAME is not defined."

In [0]:
def path_exists(path):
    """Check if a path exists"""
    try:
        dbutils.fs.ls(path)
        return True
    except:
        return False


def create_directory_if_not_exists(path):
    """Create directory if it doesn't exist"""
    if not path_exists(path):
        dbutils.fs.mkdirs(path)
        print(f"✅ Created directory: {path}")
    else:
        print(f"ℹ️  Directory already exists: {path}")


def table_exists(table_name):
    """Check if a table exists"""
    try:
        spark.table(table_name)
        return True
    except:
        return False

In [0]:
# Check if Gold ML path exists and clean if needed
print(f"\n📁 Checking Gold ML path: {GOLD_ML_PATH}")
if path_exists(GOLD_ML_PATH):
    print(f"⚠️  Path already exists. Checking if it's a valid Delta table...")
    try:
        # Try to read as Delta
        test_df = spark.read.format("delta").load(GOLD_ML_PATH)
        print(f"✅ Valid Delta table found with {test_df.count()} records")
        print(f"💡 Will overwrite existing table")
    except:
        print(f"⚠️  Path exists but is not a valid Delta table")
        print(f"🧹 Cleaning up old data...")
        dbutils.fs.rm(GOLD_ML_PATH, recurse=True)
        print(f"✅ Old data removed")
else:
    print(f"✅ Path is clear, ready to create new table")

# Create parent directory if needed
gold_ml_parent = "/".join(GOLD_ML_PATH.split("/")[:-1])
create_directory_if_not_exists(gold_ml_parent)

# Write Gold ML Delta table
print(f"\n💾 Writing Gold ML Delta table...")
try:
    df_gold_ml_final.write.format("delta").mode("overwrite").save(GOLD_ML_PATH)
    print(f"✅ Delta table written to: {GOLD_ML_PATH}")
    print(f"✅ Records written: {df_gold_ml_final.count():,}")
except Exception as e:
    print(f"❌ ERROR: Could not write Delta table")
    print(f"   Error: {str(e)}")
    print(f"\n💡 Trying to clean and retry...")
    try:
        dbutils.fs.rm(GOLD_ML_PATH, recurse=True)
        df_gold_ml_final.write.format("delta").mode("overwrite").save(GOLD_ML_PATH)
        print(f"✅ Successfully wrote Delta table after cleanup")
    except Exception as e2:
        print(f"❌ Still failed: {str(e2)}")
        raise

# Registering Delta table
print(f"\n📌 Registering Delta table as: {GOLD_ML_TABLE_NAME}")
try:
    # Ensure database exists
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}")
    print(f"✅ Database '{DATABASE_NAME}' ready")

    # Drop table if it exists (to avoid conflicts)
    spark.sql(f"DROP TABLE IF EXISTS {GOLD_ML_TABLE_NAME}")
    print(f"   Dropped existing table (if any)")

    # Create managed table
    df_for_table = spark.read.format("delta").load(GOLD_ML_PATH)
    df_for_table.write.format("delta").mode("overwrite").saveAsTable(GOLD_ML_TABLE_NAME)

    print(f"✅ Table registered successfully as '{GOLD_ML_TABLE_NAME}'!")
except Exception as e:
    print(f"⚠️  Could not create table with saveAsTable, trying alternative method...")
    try:
        # Alternative: Create external table with explicit LOCATION
        spark.sql(
            f"""
            CREATE TABLE IF NOT EXISTS {GOLD_ML_TABLE_NAME}
            USING DELTA
            LOCATION '{GOLD_ML_PATH}'
        """
        )
        print(f"✅ Table registered with LOCATION clause!")
    except Exception as e2:
        print(f"⚠️  Table registration failed: {str(e2)}")
        print(f"💡 You can still access the data directly using:")
        print(f"   spark.read.format('delta').load('{GOLD_ML_PATH}')")


📁 Checking Gold ML path: /Volumes/workspace/default/ds-capstone/gold/ml_features_experimental
✅ Path is clear, ready to create new table
✅ Created directory: /Volumes/workspace/default/ds-capstone/gold

💾 Writing Gold ML Delta table...
✅ Delta table written to: /Volumes/workspace/default/ds-capstone/gold/ml_features_experimental
✅ Records written: 2,520,650

📌 Registering Delta table as: default.gold_ml_features_experimental
✅ Database 'default' ready
   Dropped existing table (if any)
✅ Table registered successfully as 'default.gold_ml_features_experimental'!
